In [16]:
from model_util import *
import numpy as np 
import pandas as pd 
import tensorflow as tf 

## Load URLNet model 

In [2]:
word_seq_len = 200
char_seq_len = 200
ngram_seq_len = 20
chars_dict_len = 1406 
word_dict_len = 10507
ngram_dict_len = 1406
reg_lambda = 0 
emb_dim = 32

In [3]:
model = getModel(word_seq_len, char_seq_len, ngram_seq_len, chars_dict_len, word_dict_len, ngram_dict_len, reg_lambda, emb_dim)
model.load_weights("/home/youngjai/sampleData/ailab/workspace/youngjai_kwon/pml-data/model/2020-11-09/small_70k")\


shape of char_input:  (None, 200)
char-Embedding:  (None, 200, 32)
char-convOutput :  (None, 198, 256)
char - maxPool shape:  (None, 1, 256)
char-convOutput :  (None, 197, 256)
char - maxPool shape:  (None, 1, 256)
char-convOutput :  (None, 196, 256)
char - maxPool shape:  (None, 1, 256)
char-convOutput :  (None, 195, 256)
char - maxPool shape:  (None, 1, 256)
4 filter output pooled:  (None, 4, 256)
word-embedding:  (None, 200, 32)
word_flat shape:  (None, 1024)


## Input data 가져오기 

* 각각의 데이터 경로에 저장되어 있는 TFRecord 형태의 데이터 가져오기     
* tf.DataSet API 로 읽어오기 

In [4]:
train_base_dir = "/home/youngjai/sampleData/ailab/workspace/youngjai_kwon/pml-data/engineered-data/totalSet/2020-11-09/small_splitted/train.tfrecord/part-r-"
train_data_dir = [train_base_dir+str(idx).zfill(5) for idx in range(5)]

val_base_dir =  "/home/youngjai/sampleData/ailab/workspace/youngjai_kwon/pml-data/engineered-data/totalSet/2020-11-09/small_splitted/test.tfrecord/part-r-"
val_data_dir = [val_base_dir+str(idx).zfill(5) for idx in range(5)]

oop_base_dir = "/home/youngjai/sampleData/ailab/workspace/youngjai_kwon/pml-data/engineered-data/totalSet/2020-11-09/small_oop/tfrecords"
oop_data_dir = [oop_base_dir + str(idx).zfill(5) for idx in range(5)]

In [5]:
tr_dataset = input_fn(train_data_dir, word_seq_len, char_seq_len, ngram_seq_len, 10000, 256)
val_dataset = input_fn(val_data_dir, word_seq_len, char_seq_len, ngram_seq_len, 10000, 256)
oop_dataset = input_fn(oop_data_dir, word_seq_len, char_seq_len, ngram_seq_len, 10000, 256)



In [6]:
url_list=[]
label_list=[]
source=[]
for x in get_label_url_source(val_data_dir):
    url_list.append(x["url"].numpy().decode("utf-8"))
    label_list.append(np.argmax(x["label"].numpy()))
    source.append(x["urlSource"].numpy().decode("utf-8"))

engineered_char = []
engineered_ngram=[]
engineered_word=[]
for x,_ in input_small_fn(val_data_dir, 200, 200, 20, 15):
    engineered_char.append(x["char_input"].numpy())
    engineered_word.append(x["word_input"].numpy())
    engineered_ngram.append(x["ngram_input"].numpy())
    

In [7]:
len(engineered_word)


15

In [17]:
def make_dict(path):
    result={}
    tmp = pd.read_parquet(path)
    for token, index in zip(tmp.iloc[:,0], tmp.iloc[:,1]):
        result[index]=token
    return result    
word_dict = make_dict("/home/youngjai/sampleData/ailab/workspace/youngjai_kwon/pml-data/engineered-data/totalSet/2020-11-09/small_dictionary/wordTokenIndex")



In [12]:
def get_result2(model, engineered_char, engineered_ngram, engineered_word, url_list, label_list, target_label, idx_url):
    
    char_tensor = tf.convert_to_tensor(engineered_char)
    ngram_tensor = tf.convert_to_tensor(engineered_ngram)
    word_tensor = tf.convert_to_tensor(engineered_word)
    
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer("word_layer_fsize_3").output, model.output]
    )
    
    with tf.GradientTape() as tape: 
        conv_outputs, predictions = grad_model([char_tensor, word_tensor, ngram_tensor])
        loss = predictions[:, target_label]
    grads = tape.gradient(loss,conv_outputs) 
    guided_grads = tf.cast(conv_outputs > 0, "float32") * tf.cast(grads > 0, "float32") * grads  # [10, 198, 256]
    weight = tf.reduce_mean(guided_grads, axis=1)  # (10, 256)
    
    # conv_outputs : [10, 198, 256]  => [10, 256, 198]
    # tf.stack([weight],axis=2) : [10, 256, 1]
    # cam : [10, 198]
    cam = tf.reduce_sum(tf.multiply(tf.transpose(conv_outputs, perm=[0,2,1]), tf.stack([weight],axis=2)),axis=1)
    
    tmp = tf.stack([tf.stack([cam], axis=2)], axis=3)
    
    # [10, 198]  => [10,198,1,1] 으로 바꿔줌. 
    tmp = tf.image.resize(tmp, size=[200,1])
    answer = tf.squeeze(tmp, axis=[2,3]).numpy()
    
    print(url_list[idx_url], "  :  ",label_list[idx_url] )
    
    for i in range(20):
        idx2 = engineered_word[idx_url][i]
        word = word_dict.get(idx2)
        prob = answer[idx_url][i]
        print(word," ",prob,"\n")
    

In [27]:
get_result2(model, engineered_char, engineered_ngram, engineered_word, url_list, label_list, 1, 8)

http://112.17.13.225/files/3069000004ADAFEC/down1.7654.com/n/tui/tpop/tpop3/v3.0.7.19/tpop3-8.exe   :   1
http   5.537852e-05 

files   3.522598e-05 

<UNK>   3.470149e-05 

down   3.780583e-05 

com   1.9966545e-05 

n   1.5988115e-05 

tui   1.8339822e-05 

tpop   6.041335e-06 

tpop   8.285203e-06 

v   2.6826136e-05 

tpop   3.592126e-05 

exe   2.1319136e-05 

<PAD>   8.915838e-06 

<PAD>   7.430641e-06 

<PAD>   7.430641e-06 

<PAD>   7.430641e-06 

<PAD>   7.430641e-06 

<PAD>   7.430641e-06 

<PAD>   7.430641e-06 

<PAD>   7.430641e-06 

